## Рефакторинг исходной таблицы

В этой тетрадке мы немножко играем с данными андийских языков.

Данные взяты [отсюда](https://docs.google.com/spreadsheets/d/1wLwgyR_YBa5yuKg8ozN4aLbi4wvRCEYkwX1Xhnryuuw/edit#gid=0). (Предоставлены проектом *Field NLP - morpology*) 

Наша задача распарсить данные и преобразовать их в онтологию. Нужно преобразовать словарные вхождения из таблицы в формат RDF и модель Ontolex lemon. 

На питон существует rdf либа:

In [ ]:
# !pip install rdflib

RDF состоит из триплов, триплы это субъект-предикат-объект. Типа что - как_связано - с_чем. В либе rdflib вся эта сложная структура переведена в pythonic way -- у нас собирается что-то вроде контейнера\списка с 3-х элементными кортежами внутри. 

Вот пример из [документации](https://rdflib.readthedocs.io/en/stable/gettingstarted.html):

```
[
    (subject0, predicate0, object0),
    (subject1, predicate1, object1),
    ...
    (subjectN, predicateN, objectN)
 ]
```

Чтобы сразу не бросаться в огонь, я просто переведу саму таблицу с андийскими данными в pandas. А уже потом этот dataframe можно будет аккуратно переводить в rdf.

С гугл таблицами мне помог вот этот [туториал](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c)

Но к сожалению, это работает везде кроме колаба. Видимо, колаб не умеет поднимать локальные сервера.

In [4]:
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

In [1]:
andi_sheet_id = "1wLwgyR_YBa5yuKg8ozN4aLbi4wvRCEYkwX1Xhnryuuw"

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = andi_sheet_id
SAMPLE_RANGE_NAME = 'A1:S68794'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)  # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input:
        print('No data found.')

main()

df = pd.DataFrame(values_input[1:], columns=values_input[0])

In [3]:
df.head()

,,id_word,id_meaning,id,lemma,ipa,morphology,bor,annotator,pos,concepticon,meaning_ru,definition,glottocode,reference,comments,borrowing_source_word,borrowing_source_comment,"29,22%"
0,ghod1238,1,1,1,АБÁДАЛIИ,,,,George Moroz,adv,1656,никогда,нареч. 1) никогда; абадалIи йаиччи ден биттиха...,ghod1238,Saidova 2006,,,,650
1,,1,2,2,АБÁДАЛIИ,,,,George Moroz,adv,,ни за что,нареч. 1) никогда; абадалIи йаиччи ден биттиха...,ghod1238,Saidova 2006,,,,3477
2,,2,1,3,АБÁДИЛ-АБÁДАЛIИ,,,,George Moroz,adv,,см. АБÁДАЛIИ,"то же, что абáдалIи",ghod1238,Saidova 2006,,,,4344
3,,3,1,4,АБАЗИ́НА/В,,(-щуб),1,George Moroz,noun,,абазинец,I (-щуб) абазинец,ghod1238,Saidova 2006,,,,1391
4,,4,1,5,АБАЗИ́НА/Й,,(-лълъилIи),1,George Moroz,noun,,абазинец,II (-лълъилIи) абазинка,ghod1238,Saidova 2006,,,,2295
